In [1]:
import json
from tqdm.auto import tqdm
from collections import defaultdict
from copy import deepcopy

import numpy as np

from cooccurrence_matrix import CooccurrenceMatrix

/data2/cwkang/anaconda3/envs/cooccurrence_bias/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
coo_matrix = CooccurrenceMatrix()

In [3]:
from nltk.corpus import stopwords
from nltk import word_tokenize

stopword_list = stopwords.words("english")

filter = {}
for w in stopword_list:
    filter[w] = w
punctuations = {
    "?": "?",
    ":": ":",
    "!": "!",
    ".": ".",
    ",": ",",
    ";": ";"
}
filter.update(punctuations)
def filtering(text):
    if text in filter:
        return True

def text_normalization_without_lemmatization(text):
    result = []
    tokens = word_tokenize(text)
    
    for token in tokens:
        token_low = token.lower()
        if filtering(token_low):
            continue
        result.append(token_low)
    return result

In [4]:
with open("../data/LAMA_TREx/all.json", 'r') as fin:
    f_all = json.load(fin)

uid_rel_map = {}
uid_subj_map = {}
rel_subj_objects = defaultdict(set)
for example in f_all:
    subj = example['subj']
    rel = example['rel_id']
    obj = example['output']

    uid_subj_map[example['uid']] = subj
    uid_rel_map[example['uid']] = rel
    rel_subj_objects[rel+'_'+subj].add(obj.lower())
for key in rel_subj_objects:
    rel_subj_objects[key] = list(rel_subj_objects[key])

In [7]:
model_names = ['gpt_neo_125M', 'gpt_neo_125M_TREx', 'gpt_neo_1_3B', 'gpt_neo_1_3B_TREx',
               'gpt_neo_2_7B', 'gpt_neo_2_7B_TREx', 'gpt_j_6B', 'gpt_j_6B_TREx']
openai_model_names = ['text-davinci-003', 'gpt-3.5-turbo-0301', 'gpt-4-0314']

num_sections = 8
def prob_value_to_section(value):
    return min(int(np.ceil(-np.log2(value+0.000001))), num_sections - 1)
    
for model_name in model_names + openai_model_names:
    if model_name in openai_model_names:
        pred_test_filename = "../results/" + model_name + "/pred_possible_only.json"
    else:
        pred_test_filename = "../results/" + model_name + "/pred_factual_probing_test.json"

    print('='*30)
    print('='*30)
    print('Model:', model_name)
    
    with open(pred_test_filename, "r") as fin:
        preds = json.load(fin)

    print('Test')

    condprob_gt_bin_total = defaultdict(list)
    condprob_pred_bin_total = defaultdict(list)
    condprob_gt_bin_success = defaultdict(list)
    condprob_pred_bin_success = defaultdict(list)
    condprob_gt_bin_failure = defaultdict(list)
    condprob_pred_bin_failure = defaultdict(list)

    count_bin_failure = defaultdict(list)

    for pred in tqdm(preds):
        subj = uid_subj_map[pred['uid']]
        rel = uid_rel_map[pred['uid']]
        label_text = pred['label_text'].lower()
        rel_subj_object = deepcopy(rel_subj_objects[rel+'_'+subj])
        rel_subj_object.remove(label_text)

        if model_name in openai_model_names:
            pred_top_1_remove_stopwords = pred['top_1_text_remove_stopwords']
        else:
            # we remove other valid objects for a subject-relation pair other than the one we test
            for w in pred['top_100_text_remove_stopwords']:
                w = w.lower().strip()
                if w not in rel_subj_object or True:
                    pred_top_1_remove_stopwords = w
                    break

        subj = ' '.join(text_normalization_without_lemmatization(subj))
        obj_gt = ' '.join(text_normalization_without_lemmatization(label_text))
        obj_pred = ' '.join(text_normalization_without_lemmatization(pred_top_1_remove_stopwords))
        joint_freq_gt = coo_matrix.coo_count(subj, obj_gt)
        joint_freq_pred = coo_matrix.coo_count(subj, obj_pred)
        # skip if the entities are composed of more than 3 tokens, or are stopwords
        if joint_freq_gt <= 0 or joint_freq_pred <= 0:
            continue
        subj_freq = coo_matrix.count(subj)
        cond_prob_gt = joint_freq_gt / subj_freq if subj_freq > 0 else 0
        cond_prob_pred = joint_freq_pred / subj_freq if subj_freq > 0 else 0

        bin = prob_value_to_section(cond_prob_gt)

        condprob_gt_bin_total[bin].append(cond_prob_gt)
        condprob_pred_bin_total[bin].append(cond_prob_pred)
        condprob_gt_bin_total['total'].append(cond_prob_gt)
        condprob_pred_bin_total['total'].append(cond_prob_pred)
        if pred['hits@1_remove_stopwords'] > 0.5:
            condprob_gt_bin_success[bin].append(cond_prob_gt)
            condprob_pred_bin_success[bin].append(cond_prob_pred)
            condprob_gt_bin_success['total'].append(cond_prob_gt)
            condprob_pred_bin_success['total'].append(cond_prob_pred)
        else:
            condprob_gt_bin_failure[bin].append(cond_prob_gt)
            condprob_pred_bin_failure[bin].append(cond_prob_pred)
            condprob_gt_bin_failure['total'].append(cond_prob_gt)
            condprob_pred_bin_failure['total'].append(cond_prob_pred)
            count_bin_failure[bin].append((cond_prob_pred > cond_prob_gt)*1)
            count_bin_failure['total'].append((cond_prob_pred > cond_prob_gt)*1)

    # print('Total')
    # for bin in ['total'] + list(range(num_sections)):
    #     print(f"{bin} / {round(np.mean(condprob_pred_bin_total[bin]), 2)} +- {round(np.std(condprob_pred_bin_total[bin]), 2) } / {round(np.mean(condprob_gt_bin_total[bin]), 2)} +- {round(np.std(condprob_gt_bin_total[bin]), 2)} / {len(condprob_pred_bin_total[bin])}")
    print('Count in failure cases')
    for bin in ['total'] + list(range(num_sections)):
        print(f"{bin} / {int(np.mean(count_bin_failure[bin])*100)}% / {len(count_bin_failure[bin])}")
    print('Failure cases')
    for bin in ['total'] + list(range(num_sections)):
        print(f"{bin} / {round(np.mean(condprob_pred_bin_failure[bin]), 2)} +- {round(np.std(condprob_pred_bin_failure[bin]), 2) } / {round(np.mean(condprob_gt_bin_failure[bin]), 2)} +- {round(np.std(condprob_gt_bin_failure[bin]), 2)} / {len(condprob_gt_bin_failure[bin])}")
    

Model: gpt_neo_125M
Test


100%|██████████| 8856/8856 [00:01<00:00, 5466.48it/s]


Count in failure cases
total / 37% / 6094
0 / 0% / 647
1 / 14% / 2074
2 / 39% / 1487
3 / 63% / 888
4 / 79% / 481
5 / 87% / 244
6 / 94% / 147
7 / 97% / 126
Failure cases
total / 0.35 +- 0.26 / 0.48 +- 0.32 / 6094
0 / 0.37 +- 0.28 / 1.0 +- 0.0 / 647
1 / 0.37 +- 0.27 / 0.72 +- 0.14 / 2074
2 / 0.35 +- 0.24 / 0.37 +- 0.07 / 1487
3 / 0.34 +- 0.25 / 0.18 +- 0.04 / 888
4 / 0.31 +- 0.26 / 0.09 +- 0.02 / 481
5 / 0.3 +- 0.28 / 0.05 +- 0.01 / 244
6 / 0.31 +- 0.27 / 0.02 +- 0.0 / 147
7 / 0.32 +- 0.31 / 0.01 +- 0.0 / 126
Model: gpt_neo_125M_TREx
Test


100%|██████████| 8856/8856 [00:01<00:00, 5565.74it/s]


Count in failure cases
total / 31% / 3906
0 / 0% / 91
1 / 6% / 1319
2 / 18% / 935
3 / 39% / 601
4 / 60% / 414
5 / 79% / 255
6 / 89% / 164
7 / 91% / 127
Failure cases
total / 0.23 +- 0.26 / 0.4 +- 0.29 / 3906
0 / 0.4 +- 0.32 / 1.0 +- 0.0 / 91
1 / 0.24 +- 0.25 / 0.71 +- 0.14 / 1319
2 / 0.23 +- 0.24 / 0.37 +- 0.07 / 935
3 / 0.23 +- 0.26 / 0.18 +- 0.03 / 601
4 / 0.21 +- 0.24 / 0.09 +- 0.02 / 414
5 / 0.23 +- 0.28 / 0.05 +- 0.01 / 255
6 / 0.23 +- 0.27 / 0.02 +- 0.0 / 164
7 / 0.22 +- 0.28 / 0.01 +- 0.0 / 127
Model: gpt_neo_1_3B
Test


...

Count in failure cases
total / 34% / 1786
0 / 0% / 135
1 / 18% / 655
2 / 41% / 473
3 / 50% / 300
4 / 64% / 130
5 / 66% / 59
6 / 80% / 25
7 / 100% / 9
Failure cases
total / 0.35 +- 0.27 / 0.48 +- 0.3 / 1786
0 / 0.4 +- 0.3 / 1.0 +- 0.0 / 135
1 / 0.42 +- 0.28 / 0.72 +- 0.13 / 655
2 / 0.35 +- 0.26 / 0.37 +- 0.07 / 473
3 / 0.26 +- 0.23 / 0.19 +- 0.04 / 300
4 / 0.24 +- 0.24 / 0.09 +- 0.02 / 130
5 / 0.17 +- 0.21 / 0.05 +- 0.01 / 59
6 / 0.11 +- 0.11 / 0.02 +- 0.0 / 25
7 / 0.08 +- 0.07 / 0.01 +- 0.0 / 9
Model: gpt-3.5-turbo-0301
Test


100%|██████████| 4149/4149 [00:00<00:00, 5514.72it/s]


Count in failure cases
total / 26% / 1785
0 / 0% / 127
1 / 11% / 581
2 / 28% / 464
3 / 39% / 344
4 / 45% / 153
5 / 58% / 67
6 / 62% / 35
7 / 78% / 14
Failure cases
total / 0.23 +- 0.26 / 0.44 +- 0.3 / 1785
0 / 0.32 +- 0.26 / 1.0 +- 0.0 / 127
1 / 0.28 +- 0.3 / 0.71 +- 0.13 / 581
2 / 0.23 +- 0.25 / 0.37 +- 0.07 / 464
3 / 0.21 +- 0.23 / 0.18 +- 0.04 / 344
4 / 0.16 +- 0.21 / 0.09 +- 0.02 / 153
5 / 0.13 +- 0.17 / 0.05 +- 0.01 / 67
6 / 0.08 +- 0.09 / 0.02 +- 0.0 / 35
7 / 0.03 +- 0.03 / 0.01 +- 0.0 / 14
Model: gpt-4-0314
Test


100%|██████████| 4149/4149 [00:00<00:00, 5371.08it/s]

Count in failure cases
total / 28% / 1748
0 / 0% / 121
1 / 15% / 573
2 / 25% / 468
3 / 43% / 355
4 / 52% / 136
5 / 56% / 55
6 / 80% / 26
7 / 92% / 14
Failure cases
total / 0.26 +- 0.3 / 0.45 +- 0.3 / 1748
0 / 0.33 +- 0.28 / 1.0 +- 0.0 / 121
1 / 0.29 +- 0.33 / 0.72 +- 0.13 / 573
2 / 0.26 +- 0.31 / 0.37 +- 0.07 / 468
3 / 0.23 +- 0.26 / 0.19 +- 0.04 / 355
4 / 0.26 +- 0.31 / 0.09 +- 0.02 / 136
5 / 0.2 +- 0.25 / 0.05 +- 0.01 / 55
6 / 0.17 +- 0.19 / 0.02 +- 0.0 / 26
7 / 0.11 +- 0.19 / 0.01 +- 0.0 / 14
